In [1]:
from tensorflow.keras.datasets import mnist
import numpy as np

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))  # adapt this if using `channels_first` image data format

noise_factor = 0.5
x_train_noisy = x_train + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_train.shape) 
x_test_noisy = x_test + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=x_test.shape) 

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

11493376/11490434 [==============================] - 0s 0us/step


In [2]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K

input_img = Input(shape=(28, 28, 1))  # adapt this if using `channels_first` image data format

x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (7, 7, 32)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [5]:
history=autoencoder.fit(x_train_noisy, x_train,
                epochs=100,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_noisy, x_test))

Epoch 1/100
469/469 [==============================] - 157s 335ms/step - loss: 0.4435 - val_loss: 0.4413
Epoch 2/100
469/469 [==============================] - 157s 336ms/step - loss: 0.4326 - val_loss: 0.4296
Epoch 3/100
469/469 [==============================] - 157s 335ms/step - loss: 0.4202 - val_loss: 0.4162
Epoch 4/100
469/469 [==============================] - 156s 333ms/step - loss: 0.4048 - val_loss: 0.3975
Epoch 5/100
469/469 [==============================] - 158s 336ms/step - loss: 0.3855 - val_loss: 0.3781
Epoch 6/100
469/469 [==============================] - 157s 334ms/step - loss: 0.3652 - val_loss: 0.3561
Epoch 7/100
469/469 [==============================] - 157s 335ms/step - loss: 0.3417 - val_loss: 0.3303
Epoch 8/100
469/469 [==============================] - 153s 325ms/step - loss: 0.3150 - val_loss: 0.3024
Epoch 9/100
469/469 [==============================] - 154s 329ms/step - loss: 0.2882 - val_loss: 0.2766
Epoch 10/100
469/469 [==============================] -

In [6]:
decoded_imgs = autoencoder.predict(x_test_noisy)

print(decoded_imgs.shape)

(10000, 28, 28, 1)


New model is trained using noise reduced decoded images for the classification and it's accuarcies can be seen as follows after each epoch

In [12]:
# Importing the required Keras modules containing model and layers
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input,Dense, Conv2D, Dropout, Flatten, MaxPooling2D
# Creating a Sequential Model and adding the layers

input_shape = (28, 28, 1)
model = tensorflow.keras.Sequential(
    [
        Input(shape=input_shape),
        Conv2D(32, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(64, kernel_size=(3, 3), activation="relu"),
        MaxPooling2D(pool_size=(2, 2)),
        
        Flatten(),
        
        Dense(128, activation=tf.nn.relu),
        Dropout(0.5),
        Dense(10, activation=tf.nn.softmax),
    ]
)

In [15]:
from tensorflow.keras.datasets import mnist
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshaping the array to 4-dims so that it can work with the Keras API
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)
# Making sure that the values are float so that we can get decimal points after division
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# Normalizing the RGB codes by dividing it to the max RGB value.
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])


# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

x_train shape: (60000, 28, 28, 1)
Number of images in x_train 60000
Number of images in x_test 10000


In [16]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(decoded_imgs, y_test, epochs=10, validation_split=0.1)

Epoch 1/10
282/282 [==============================] - 9s 31ms/step - loss: 0.8308 - accuracy: 0.7282 - val_loss: 0.4024 - val_accuracy: 0.8770
Epoch 2/10
282/282 [==============================] - 9s 30ms/step - loss: 0.4318 - accuracy: 0.8610 - val_loss: 0.3350 - val_accuracy: 0.8950
Epoch 3/10
282/282 [==============================] - 8s 29ms/step - loss: 0.3709 - accuracy: 0.8831 - val_loss: 0.3172 - val_accuracy: 0.9060
Epoch 4/10
282/282 [==============================] - 8s 29ms/step - loss: 0.3244 - accuracy: 0.8954 - val_loss: 0.2888 - val_accuracy: 0.9140
Epoch 5/10
282/282 [==============================] - 8s 29ms/step - loss: 0.2905 - accuracy: 0.9059 - val_loss: 0.2896 - val_accuracy: 0.9060
Epoch 6/10
282/282 [==============================] - 8s 29ms/step - loss: 0.2703 - accuracy: 0.9124 - val_loss: 0.2979 - val_accuracy: 0.9080
Epoch 7/10
282/282 [==============================] - 8s 29ms/step - loss: 0.2488 - accuracy: 0.9197 - val_loss: 0.2814 - val_accuracy: 0.9130